In [1]:
import json
from dataclasses import dataclass
from pathlib import Path

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

%load_ext autoreload
%autoreload 2

In [2]:
class Education(BaseModel):
    academic_degrees: list[str] = Field(description="List of relevant academic degrees")
    weightage: float = Field(default=0.15, description="Weightage percentage between 10-15%")


class ProfessionalExperience(BaseModel):
    total_years: int = Field(description="Total years of experience in relevant field")
    project_experience: list[str] = Field(description="Specific project-related experience matching project scope")
    leadership_roles: list[str] = Field(description="Experience in leadership positions")
    weightage: float = Field(default=0.3, description="Weightage percentage between 25-30%")


class PECRegistration(BaseModel):
    registration_type: str = Field(description="Type of PEC registration (PE/RE)")
    certifications: list[str] = Field(description="Additional professional certifications")
    weightage: float = Field(default=0.1, description="Weightage percentage between 5-10%")


class ProjectExperience(BaseModel):
    similar_projects: list[str] = Field(description="Experience in similar projects")
    completed_projects: int = Field(description="Number of completed projects")
    project_scale: list[str] = Field(description="Scale of completed projects")
    weightage: float = Field(default=0.25, description="Weightage percentage between 20-25%")


class Training(BaseModel):
    professional_training: list[str] = Field(description="Professional training and workshops")
    cpd_points: int = Field(description="CPD points as per PEC requirements")
    weightage: float = Field(default=0.05, description="Weightage percentage between 2.5-5%")


class Publications(BaseModel):
    publications: list[str] = Field(description="Relevant publications and technical papers")
    research_work: list[str] = Field(description="Research work in relevant field")
    weightage: float = Field(default=0.05, description="Weightage percentage between 2.5-5%")


class CVEvaluationCriteria(BaseModel):
    education: Education
    professional_experience: ProfessionalExperience
    pec_registration: PECRegistration
    project_experience: ProjectExperience
    training: Training
    publications: Publications | None = None


class PositionCriteria(BaseModel):
    position_title: str
    position_description: str
    number_of_experts: int
    position_criteria: CVEvaluationCriteria


class Evaluation(BaseModel):
    score: float
    comments: str
    criteria: str = Field(
        description=(
            "The specific, measurable requirements used to evaluate this category. List each requirement "
            "with its exact threshold or value. For example:\n"
            "- 'Minimum 15 years experience in water supply projects'\n"
            "- 'Masters or PhD in Civil/Environmental Engineering'\n"
            "- 'At least 3 projects over $50M budget as team lead'\n"
            "Do not include evaluation methodology or assessment language. Focus only on the concrete "
            "requirements that the candidate was measured against."
        )
    )


class PositionEvaluation(BaseModel):
    position_title: str
    education: Evaluation
    professional_experience: Evaluation
    pec_registration: Evaluation
    project_experience: Evaluation
    training: Evaluation
    publications: Evaluation


@dataclass
class RFP:
    datasheet: str
    terms_of_reference: str

    def __str__(self) -> str:
        return (
            f"\n<RFP>\n\n"
            f"<datasheet>\n{self.datasheet}\n</datasheet>\n\n"
            f"<terms_of_reference>\n{self.terms_of_reference}\n</terms_of_reference>\n\n"
            f"</RFP>"
        )


In [3]:
criteria_agent = Agent(
    "google-gla:gemini-2.0-flash-001",
    result_type=list[PositionCriteria],
    system_prompt=(
        "You are an expert in analyzing Request for Proposals (RFPs) for engineering projects. "
        "Your task is to extract and structure the personnel requirements and evaluation criteria "
        "for key positions from the RFP.\n\n"
        "First, identify key positions using these guidelines:\n"
        "1. Positions explicitly marked as 'key positions' in the RFP, if any\n"
        "2. If no explicit designation, identify key positions based on:\n"
        "   - Senior management or leadership roles\n"
        "   - Positions requiring specialized technical expertise\n"
        "   - Roles critical to project delivery\n\n"
        "For each identified key position:\n"
        "1. Identify the position title, number of experts required, and description\n"
        "2. Extract specific qualification requirements and evaluation criteria\n"
        "3. Structure position-specific requirements into weighted categories\n\n"
        "For each position, focus on identifying:\n"
        "- Required educational qualifications and degrees\n"
        "- Minimum years of experience specific to the role\n"
        "- Technical expertise needed for the position\n"
        "- Required project experience (size, complexity, similarity)\n"
        "- Mandatory certifications and registrations (esp. PEC requirements)\n"
        "- Any additional requirements like publications or training\n\n"
        "For each position, assign weightages based on the emphasis in the RFP, staying within these ranges:\n"
        "- Education & Qualification: 10-15%\n"
        "- Professional Experience: 25-30%\n"
        "- PEC Registration & Certifications: 5-10%\n"
        "- Relevant Project Experience: 20-25%\n"
        "- Training & CPD: 2.5-5%\n"
        "- Publications & Research: 2.5-5%\n\n"
        "The total weightage for each position must sum to 100%. Return a list of structured criteria "
        "for each key position in the required format."
    ),
    deps_type=RFP,
)


@criteria_agent.system_prompt(dynamic=True)
def system_prompt(ctx: RunContext[RFP]) -> str:
    return str(ctx.deps)


evaluation_agent = Agent(
    "google-gla:gemini-2.0-flash-001",
    result_type=PositionEvaluation,
    system_prompt=(
        "You are an expert in evaluating engineering personnel based on their qualifications and experience. "
        "Your task is to evaluate a candidate's CV against specific position criteria and provide detailed scoring "
        "and feedback for each evaluation category.\n\n"
        "For each category:\n"
        "1. Compare the CV against the specific requirements\n"
        "2. Assign a score from 0-100 based on how well they meet the criteria\n"
        "3. Provide specific comments justifying the score\n"
        "4. List the concrete requirements that were evaluated against (e.g., 'Required: 10 years experience, Masters degree in Engineering')\n\n"
        "Scoring guidelines:\n"
        "- 90-100: Exceeds requirements significantly\n"
        "- 80-89: Fully meets requirements\n"
        "- 70-79: Meets most requirements\n"
        "- 60-69: Meets basic requirements\n"
        "- Below 60: Does not meet minimum requirements\n\n"
        "For each evaluation category, focus on concrete, measurable requirements:\n\n"
        "Education:\n"
        "- Specific required degrees and fields\n"
        "- Minimum education level needed\n"
        "- Required specializations\n\n"
        "Professional Experience:\n"
        "- Minimum years of experience required\n"
        "- Specific types of projects/roles needed\n"
        "- Required leadership level and team size\n\n"
        "PEC Registration:\n"
        "- Required registration type (PE/RE)\n"
        "- Mandatory certifications\n"
        "- Specific professional memberships\n\n"
        "Project Experience:\n"
        "- Minimum number of similar projects\n"
        "- Required project budget/scale thresholds\n"
        "- Specific project types or sectors\n\n"
        "Training:\n"
        "- Required certifications or courses\n"
        "- Minimum CPD points needed\n"
        "- Specific technical training requirements\n\n"
        "Publications:\n"
        "- Required number of publications\n"
        "- Specific types of research work\n"
        "- Required technical contributions\n\n"
        "For each category, your evaluation must include:\n"
        "1. The specific requirements being evaluated against\n"
        "2. How the candidate meets or falls short of each requirement\n"
        "3. Quantitative measures where applicable (years, numbers, levels)\n"
        "Avoid vague assessment language - focus on concrete requirements and measurements."
    ),
    deps_type=PositionCriteria,
)


@evaluation_agent.system_prompt(dynamic=True)
def evaluation_system_prompt(ctx: RunContext[PositionCriteria]) -> str:
    criteria = ctx.deps
    return (
        f"<position>\n{criteria.position_title}\n</position>\n\n"
        f"<position_description>\n{criteria.position_description}\n</position_description>\n\n"
        f"<evaluation_criteria>\n{json.dumps(criteria.position_criteria.model_dump(), indent=2)}\n</evaluation_criteria>"
    )


In [4]:
rfp_data = json.loads(Path("tor_datasheet.json").read_text())
rfp_data

{'datasheet': '###### Section 2. Instructions to Consultants\n\n|Col1|E. Data Sheet|\n|---|---|\n|ITC Reference|A. General|\n|1 (b)|Islamic Republic of Pakistan|\n|1 (o)|Not Applicable|\n|2.1|Name of the Client: Punjab Rural Municipal Services Company Method of selection: Quality & Cost Based Selection (QCBS) as per World Bank’s Procurement Regulations (available on www.worldbank.org)|\n|2.2|Financial Proposal to be submitted together with Technical Proposal: Yes The name of the assignment is: Selection of Engineering, Design & Construction Supervision Consultant for Cluster South – II|\n|2.3|A pre-proposal conference will be held on 22nd April, 2024, 1100 hours at CONFERENCE ROOM Punjab Rural Municipal Services Company, 5th Floor, KD Plaza, MM Alam Road, near Layers Bakers, Lahore, Pakistan. Join Zoom Meeting https://us06web.zoom.us/j/81888202479?pwd=F0n4Leb5hW9cvavXWcKsmt hpfbj45V.1 Meeting ID: 818 8820 2479 Passcode: 995446 Contact Person: Manager Procurement PUNJAB RURAL MUNICIPAL 

In [5]:
rfp = RFP(datasheet=rfp_data["datasheet"], terms_of_reference=rfp_data["terms_of_reference"])

In [6]:
criteria = await criteria_agent.run("Extract", deps=rfp)


/tmp/ipykernel_19570/1996902261.py:1: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  criteria = await criteria_agent.run("Extract", deps=rfp)


In [7]:
criteria.data

[PositionCriteria(position_title='Team Leader (Water Supply & Sanitation Expert)', position_description='Lead a multidisciplinary team of water supply and sanitation professionals to plan, design, implement, and evaluate projects and programs aimed at improving access to safe water and sanitation services. Provide technical leadership and guidance on water supply and sanitation engineering, hygiene promotion, and behavior change interventions. Oversee the development and implementation of comprehensive water supply and sanitation strategies, ensuring alignment with project goals, objectives, and donor requirements. Coordinate with government agencies, non-governmental organizations (NGOs), community leaders, and other stakeholders to mobilize support and resources for water supply and sanitation initiatives. Conduct needs assessments, feasibility studies, and technical evaluations to identify priority areas and interventions for water supply and sanitation improvement. Manage project b

In [ ]:
Path("position_criteria.json").write_text(json.dumps([crit.model_dump() for crit in criteria.data], indent=2))

38820

In [6]:
position_criteria = PositionCriteria(**json.loads(Path("position_criteria.json").read_text())[0])

In [7]:
position_criteria

PositionCriteria(position_title='Team Leader (Water Supply & Sanitation Expert)', position_description='Lead a multidisciplinary team of water supply and sanitation professionals to plan, design, implement, and evaluate projects and programs aimed at improving access to safe water and sanitation services. Provide technical leadership and guidance on water supply and sanitation engineering, hygiene promotion, and behavior change interventions. Oversee the development and implementation of comprehensive water supply and sanitation strategies, ensuring alignment with project goals, objectives, and donor requirements. Coordinate with government agencies, non-governmental organizations (NGOs), community leaders, and other stakeholders to mobilize support and resources for water supply and sanitation initiatives. Conduct needs assessments, feasibility studies, and technical evaluations to identify priority areas and interventions for water supply and sanitation improvement. Manage project bu

In [8]:
cv = """
Dr. Sarah Chen has 15 years of experience in water and sanitation engineering (vs required 20 years). She holds a Master's in Civil Engineering from the University of Michigan with a focus on Environmental Engineering. While she hasn't directly led World Bank projects, she has managed several large-scale municipal water infrastructure projects in Southeast Asia, including a $50M urban water supply upgrade in Manila and wastewater treatment facilities in Bangkok.

Her technical expertise covers water supply system design, sanitation infrastructure planning, and public health engineering. She's registered with the Professional Engineers Board and has supervised teams of 10-15 engineers. Her project portfolio includes feasibility studies, technical evaluations, and stakeholder engagement, though primarily with regional rather than international funding bodies.

Key strengths:
- Education matches requirements (M.Sc. Civil/Environmental Engineering)
- Strong technical background in water/sanitation
- Team leadership experience
- Professional registration

Areas falling short:
- 15 years experience (vs required 20)
- Limited World Bank/foreign funded project experience
- No significant publications or research work
"""

In [9]:
evaluation = await evaluation_agent.run(f"Evaluate this candidate: {cv}", deps=position_criteria)

/tmp/ipykernel_22901/3167658008.py:1: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  evaluation = await evaluation_agent.run(f"Evaluate this candidate: {cv}", deps=position_criteria)


In [10]:
Path("evaluation.json").write_text(json.dumps(evaluation.data.model_dump(), indent=2))

2033

In [ ]:
1. 3530211577344
2. rimalminhaj20@gmail.com 
3. House 1216-A, Street 28, Phase 4, Bahria Town, Islamabad, Pakistan
4. +923225065566, Jazz